In [1]:
import torch
import torchdrug as td
from torchdrug import data,datasets
from torchdrug import core, models, tasks
%matplotlib inline

In [2]:
dataset = datasets.ClinTox("~/molecule-datasets/")
lengths = [int(0.8 * len(dataset)), int(0.1 * len(dataset))]
lengths += [len(dataset) - sum(lengths)]
train_set, valid_set, test_set = torch.utils.data.random_split(dataset, lengths)

Constructing molecules from SMILES:   0%|          | 0/1484 [00:00<?, ?it/s]/home/jxy/anaconda3/envs/chemprop_1119/lib/python3.9/site-packages/torchdrug-0.1.2-py3.9.egg/torchdrug/data/feature.py:37: UserWarning: Unknown value `*`
  warnings.warn("Unknown value `%s`" % x)
RDKit ERROR: [18:21:35] Explicit valence for atom # 0 N, 5, is greater than permitted
[18:21:35] Explicit valence for atom # 0 N, 5, is greater than permitted
/home/jxy/anaconda3/envs/chemprop_1119/lib/python3.9/site-packages/torchdrug-0.1.2-py3.9.egg/torchdrug/data/feature.py:37: UserWarning: Unknown value `Tc`
  warnings.warn("Unknown value `%s`" % x)
/home/jxy/anaconda3/envs/chemprop_1119/lib/python3.9/site-packages/torchdrug-0.1.2-py3.9.egg/torchdrug/data/feature.py:37: UserWarning: Unknown value `Fe`
  warnings.warn("Unknown value `%s`" % x)
/home/jxy/anaconda3/envs/chemprop_1119/lib/python3.9/site-packages/torchdrug-0.1.2-py3.9.egg/torchdrug/data/feature.py:37: UserWarning: Unknown value `Al`
  warnings.warn("Unk

In [6]:
model = models.GIN(input_dim=dataset.node_feature_dim,hidden_dims=[256,256,256,256],short_cut=True,batch_norm=True,concat_hidden=True)
task = tasks.PropertyPrediction(model,task=dataset.tasks,criterion="bce",metric=("auprc","auroc"))

In [7]:
optimizer = torch.optim.Adam(task.parameters(),lr=1e-3)
solver = core.Engine(task,train_set,valid_set,test_set,optimizer,gpus=[0],batch_size=1024)
solver.train(num_epoch=200)
solver.evaluate("valid")

18:24:50   Preprocess training set
18:24:50   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
18:24:50   Epoch 0 begin
18:24:50   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
18:24:50   binary cross entropy: 4.77177
18:24:50   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
18:24:50   Epoch 0 end
18:24:50   duration: 0.16 secs
18:24:50   speed: 12.64 batch / sec
18:24:50   ETA: 31.49 secs
18:24:50   max GPU memory: 726.1 MiB
18:24:50   ------------------------------
18:24:50   average binary cross entropy: 3.56844
18:24:50   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
18:24:50   Epoch 1 begin
18:24:51   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
18:24:51   Epoch 1 end
18:24:51   duration: 0.09 secs
18:24:51   speed: 21.51 batch / sec
18:24:51   ETA: 24.87 secs
18:24:51   max GPU memory: 719.9 MiB
18:24:51   ------------------------------
18:24:51   average binary cross entropy: 2.65538
18:24:51   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
18:24:51   Epoch 2 begin
18:24:51   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
18:24:51   Epoch 2 end
18:24:51   duration: 0.09 secs
18:24:51   

{'auprc [FDA_APPROVED]': tensor(0.9726, device='cuda:0'),
 'auprc [CT_TOX]': tensor(0.4738, device='cuda:0'),
 'auroc [FDA_APPROVED]': tensor(0.8043, device='cuda:0'),
 'auroc [CT_TOX]': tensor(0.7560, device='cuda:0')}